> ## **Plotly**

### Allowed functions : ```import pandas as pd, import sqlite3, import plotly.graph_objects as go, import numpy as np```

In [1]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import numpy as np

In [10]:
conn=sqlite3.connect('../data/checking-logs.sqlite')
query='''
select uid, numTrials,timestamp
from checker
where uid not like 'admin%'
and status == 'ready'
and labname== 'project1'
'''

df = pd.read_sql(query, conn, parse_dates=['timestamp'])
df

,uid,numTrials,timestamp
0,user_4,1,2020-04-17 05:19:02.744528
1,user_4,2,2020-04-17 05:22:45.549397
2,user_4,3,2020-04-17 05:34:24.422370
3,user_4,4,2020-04-17 05:43:27.773992
4,user_4,5,2020-04-17 05:46:32.275104
...,...,...,...
946,user_19,26,2020-05-15 10:22:39.698523
947,user_19,27,2020-05-15 10:22:46.248162
948,user_19,28,2020-05-15 10:23:18.043212
949,user_28,27,2020-05-15 10:38:14.430013


In [17]:
df = df.sort_values(['uid', 'timestamp'])
df['step'] = df.groupby('uid').cumcount() + 1
df['date'] = df['timestamp'].dt.date
df_grouped = df.groupby(['date', 'uid']).count()['timestamp'].unstack().fillna(0).cumsum()
frames = []
users = df_grouped.columns.tolist()
dates = df_grouped.index.tolist()

In [18]:
initial_data = [go.Scatter(x=[dates[0]], y=[df_grouped.iloc[0][uid]], name=uid, mode="lines+markers") for uid in users]

In [19]:
for i in range(1, len(dates)):
    curr_date = dates[i]
    frame_data = [go.Scatter(x=dates[:i+1], y=df_grouped.iloc[:i+1][uid], name=uid, mode="lines") for uid in users]
    frames.append(go.Frame(data=frame_data, name=str(curr_date)))

In [20]:
fig = go.Figure(
    data=initial_data,
    layout=go.Layout(
        title="Dynamic of commits per user in project1",
        xaxis=dict(title='Date', range=[min(dates), max(dates)]),
        yaxis=dict(title='Cumulative Commits', range=[0, df_grouped.max().max() + 10]),
        updatemenus=[{
            "type": "buttons",
            "buttons": [{
                "label": "Play",
                "method": "animate",
                "args": [None, {"frame": {"duration": 200, "redraw": True}, "fromcurrent": True}]
            }]
        }]
    ),
    frames=frames
)

fig.show()

In [21]:
conn.close()